In [1]:
import sys
sys.path.append("..")

import pandas as pd
import ast
import seaborn as sns
import matplotlib.pyplot as plt
from reader import generic_reader
from inflation import read_cpi_by_year, convert_usd

In [2]:
cpi_by_year = read_cpi_by_year("inflation_data/cpiai.csv")

## Preparation of movie and persons data

This notebook outlines the data preparation process, which begins with preliminary cleaning.  
Since we'll be working with different parts of the DataFrame, more detailed cleaning will be performed in the relevant sections of the project

Our dataset of movies, are movies from the USA, from the periode 1/1/2000 to 31/12/2023 and with a vote count of atleast 10

In [3]:
movies = generic_reader.read_csv_file_to_data_frame("movie_data/movies.csv")
print(movies.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27290 entries, 0 to 27289
Data columns (total 31 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   adult                   27290 non-null  bool   
 1   backdrop_path           24958 non-null  object 
 2   budget                  27290 non-null  int64  
 3   homepage                11192 non-null  object 
 4   movie_id                27290 non-null  int64  
 5   imdb_id                 27150 non-null  object 
 6   origin_country          27290 non-null  object 
 7   original_language       27290 non-null  object 
 8   original_title          27289 non-null  object 
 9   overview                27275 non-null  object 
 10  popularity              27290 non-null  float64
 11  poster_path             27169 non-null  object 
 12  production_countries    27290 non-null  object 
 13  release_date            27290 non-null  object 
 14  revenue                 27290 non-null

We make a new dataframe keeping only the columns we want to use later

In [4]:
columns_to_keep = ['budget', 'movie_id', 'original_title', 'overview', 'release_date', 'revenue', 'runtime', 'vote_average', 'vote_count', 'director_person_ids', 'genre_ids', 'collection_id', 'cast_person_ids', 'cast_credit_ids', 'crew_person_ids', 'crew_credit_ids']
df_movies = movies[columns_to_keep].copy()

df_movies['release_date'] = pd.to_datetime(df_movies['release_date'], errors='coerce')
df_movies['genre_ids'] = movies['genre_ids'].apply(ast.literal_eval)
df_movies

,budget,movie_id,original_title,overview,release_date,revenue,runtime,vote_average,vote_count,director_person_ids,genre_ids,collection_id,cast_person_ids,cast_credit_ids,crew_person_ids,crew_credit_ids
0,0,515728,Hitch,Two friends are on a road trip and a one-sided...,2000-01-01,0,18,4.900,10,[131388],[18],NaN,"[1230580, 2030046]","['5ac28e640e0a260c140239f0', '5ae16159c3a36876...","[1434896, 131388, 131388, 131388, 131388, 1360...","['6454f309c044290143e43376', '6454f31187a27a01..."
1,0,300236,Carnage: The Legend of Quiltface,Four students set out for the barren Nevada de...,2000-01-01,0,70,2.500,10,[103123],[27],NaN,"[98740, 99106, 98276, 1685427, 1771744, 177174...","['58bf16e6925141608406b270', '6251f3b9a6c10432...","[103123, 98868, 103123, 1001648, 103123, 10016...","['62c5bceaf794ad00bf5a8867', '62c5bce19ba86a00..."
2,0,96716,The Bumblebee Flies Anyway,An amnesiac youth tries to piece together his ...,2000-01-01,0,95,6.200,30,[126537],"[18, 10749]",NaN,"[109, 21197, 16407, 1223778, 38581, 303197, 56...","['52fe49be9251416c750d1f8f', '52fe49be9251416c...","[1534680, 2556479, 1516275, 68126, 1516278, 27...","['60c1e4cb39a45d0040c5cd4b', '60c1e503960cde00..."
3,0,71618,After Sex,A group of attractive women get together for a...,2000-01-01,0,96,5.000,27,[176312],"[35, 18, 10749]",NaN,"[170638, 12519, 15110, 3208, 51670, 61962, 117...","['53d9db080e0a2652f0001583', '52fe483ec3a36848...","[176312, 1470931, 33008, 17210, 17211, 954441,...","['52fe483ec3a368484e0ef37f', '5564e59bc3a36874..."
4,0,66131,A Constant Forge,"One of the great mavericks of cinema, John Cas...",2000-01-01,0,200,6.600,13,[544690],[99],NaN,"[11147, 5950, 10556, 10127, 856, 2314, 1629458...","['5a9dfc9e0e0a2671fb009de8', '5a9dfc11c3a36842...","[544690, 3399184, 544690, 961119, 3399183, 339...","['52fe472bc3a368484e0b89d9', '61f15d7dcd204600..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27285,40000000,823452,The Boys in the Boat,The triumphant underdog story of the Universit...,2023-12-25,54401583,124,7.233,311,[1461],"[18, 36]",NaN,"[33192, 1371041, 27172, 1522758, 1780950, 2725...","['6205b7c5ce5d8200c83376f3', '61c21146904f6d00...","[3069889, 1461, 17148, 1500871, 1578875, 18195...","['6088693d84448e007932a40c', '6088695754a09800..."
27286,90000000,558915,The Color Purple,A decades-spanning tale of love and resilience...,2023-12-25,68919191,141,7.049,326,[1507725],[18],NaN,"[165909, 40036, 1075037, 91671, 1154054, 34767...","['61fc8db841429100a2189044', '61f966bfeee18600...","[3511353, 4445624, 4445622, 73350, 2167926, 21...","['62b89b15a61de103b61eed9b', '65876f514772155a..."
27287,0,1215278,Hell Camp: Teen Nightmare,Out-of-control teens across America were sent ...,2023-12-27,0,90,5.663,46,[2283876],[99],NaN,"[5332322, 38406]","['67e5d2703e65c8ea88ba1e0f', '67e60712421eb8c3...","[2283876, 3248862, 2060153, 4208975, 1414935, ...","['65734ffc1c635b00c3a9bfbe', '67e5d17f421eb8c3..."
27288,0,1156189,Ryuichi Sakamoto: Opus,"""Ars longa, vita brevis"" – art is long, life i...",2023-12-27,0,103,7.861,18,[2010321],"[10402, 99]",NaN,[11382],['64bfde5d8c0a4800aeefb198'],"[3056, 1610252, 2247870, 3406563, 4240016, 152...","['64ec9e951feac100fe5e313e', '64ec9e335258ae01..."


Removing animation and documentary from the dataset as we'll be fociusing on motion pictures

In [5]:
df_movies = df_movies[df_movies['genre_ids'].map({16, 99}.isdisjoint)]

In [6]:
df_movies.shape

(21665, 16)

In [7]:
df_movies.describe()

,budget,movie_id,release_date,revenue,runtime,vote_average,vote_count,collection_id
count,2.166500e+04,2.166500e+04,21665,2.166500e+04,21665.000000,21665.000000,21665.000000,2.457000e+03
mean,7.428951e+06,3.246050e+05,2013-08-09 06:30:49.434572032,1.823794e+07,91.598569,5.642227,590.707085,4.665931e+05
min,0.000000e+00,2.200000e+01,2000-01-01 00:00:00,0.000000e+00,0.000000,1.300000,10.000000,1.000000e+01
25%,0.000000e+00,4.902100e+04,2008-09-09 00:00:00,0.000000e+00,85.000000,4.940000,18.000000,1.238000e+05
50%,0.000000e+00,2.734810e+05,2014-06-29 00:00:00,0.000000e+00,91.000000,5.748000,41.000000,4.033740e+05
75%,5.000000e+05,5.149750e+05,2018-10-19 00:00:00,0.000000e+00,101.000000,6.400000,203.000000,7.208790e+05
max,5.839000e+08,1.472042e+06,2023-12-31 00:00:00,2.923706e+09,999.000000,10.000000,38021.000000,1.553427e+06
std,2.530916e+07,2.971773e+05,NaN,9.149137e+07,24.346545,1.104018,2013.664644,3.790781e+05


We will be focusing on movies with a runtime of atleast 60 min but no more than four hours (240 mins)

In [8]:
df_movies = df_movies[(df_movies['runtime'] >= 60) & (df_movies['runtime'] <= 240)]

Dropping movies with no budget data

In [9]:
df_movies.shape

(20489, 16)

In [10]:
zero_budget_count = (df_movies['budget'] == 0).sum()
print(f"Rows with budget = 0: {zero_budget_count}")

df_movies = df_movies[df_movies['budget'] != 0].copy()

print(f"Remaining rows after removing budget = 0: {len(df_movies)}")


Rows with budget = 0: 14289
Remaining rows after removing budget = 0: 6200


Adding two new columns budget_2023_usd and revenue_2023_usd which adjust for inflation. 

In [11]:
df_movies['budget_2023_usd'] = df_movies.apply(
    lambda row: convert_usd(
        original_amount=row['budget'],
        original_year=row['release_date'].year,
        new_year=2023,
        cpi_by_year=cpi_by_year
    ),
    axis=1
)

# Convert revenue to 2023 USD
df_movies['revenue_2023_usd'] = df_movies.apply(
    lambda row: convert_usd(
        original_amount=row['revenue'],
        original_year=row['release_date'].year,
        new_year=2023,
        cpi_by_year=cpi_by_year
    ),
    axis=1
)

The preliminary cleaning of movies leave us with dataframe of 6200 movies

In [12]:
df_movies.shape

(6200, 18)

Export cleaned movie data for later analysis

In [13]:
df_movies.to_csv("movie_data/movies_clean.csv", index=False)

## Cleaning data for actors/actress

The actors/actresses dataframe will be cleaned more thoroughly

In [14]:
df_persons = generic_reader.read_csv_file_to_data_frame("movie_data/persons.csv")
print(df_persons.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 104928 entries, 0 to 104927
Data columns (total 14 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   adult                 104928 non-null  bool   
 1   also_known_as         104928 non-null  object 
 2   biography             35263 non-null   object 
 3   birthday              40091 non-null   object 
 4   deathday              3729 non-null    object 
 5   gender                104928 non-null  int64  
 6   homepage              10229 non-null   object 
 7   person_id             104928 non-null  int64  
 8   imdb_id               81944 non-null   object 
 9   known_for_department  104928 non-null  object 
 10  name                  104928 non-null  object 
 11  place_of_birth        41492 non-null   object 
 12  popularity            104928 non-null  float64
 13  profile_path          61033 non-null   object 
dtypes: bool(1), float64(1), int64(2), object(10)
memory 

Keeping only the columns needed for analysis

In [15]:
columns_to_keep = ['name', 'birthday', 'deathday', 'gender', 'person_id']
df_persons = df_persons[columns_to_keep].copy()
df_persons['birthday'] = pd.to_datetime(df_persons['birthday'])
df_persons['deathday'] = pd.to_datetime(df_persons['deathday'])
df_persons

,name,birthday,deathday,gender,person_id
0,George Lucas,1944-05-14,NaT,2,1
1,Mark Hamill,1951-09-25,NaT,2,2
2,Harrison Ford,1942-07-13,NaT,2,3
3,Carrie Fisher,1956-10-21,2016-12-27,1,4
4,Thomas Lyons,NaT,NaT,2,1835010
...,...,...,...,...,...
104923,Kristen Doute,NaT,NaT,1,1015800
104924,Johnny Melville,NaT,NaT,2,1048560
104925,Eric da Silva,1977-06-30,NaT,2,1048570
104926,David Kaye,1988-08-04,NaT,2,1048574


Dropping actors/actresses with no data for birthdays

In [16]:
df_persons = df_persons[pd.notna(df_persons['birthday'])].copy()

In [17]:
birthday_nat_count = df_persons['birthday'].isna().sum()

print(f"'NaT' in 'birthday' column: {birthday_nat_count}")

'NaT' in 'birthday' column: 0


This leave actors/actresses dataframe with 400091 persons

In [18]:
df_persons.shape

(40091, 5)

In [19]:
df_persons.to_csv("movie_data/persons_clean.csv", index=False)